In [5]:
# pip install dash

In [6]:
# pip install dash-bootstrap-components


In [8]:
import pandas as pd
df = pd.read_csv('dummy_data.csv')

df.Date=pd.to_datetime(df.Date)

df.sample(5)

,Record No.,Date,Name of the student,UCID Number,Class,Mobile Number,Sanctioned Amount,Description of Scheme,Remark
1983,1984,2024-03-12,Lalbahadur Upadhyay,2009233937,EXTC,9876538931,1417,"Membership - IET, IEEE, CSI, ASM",Top scorer in the recent quiz competition.
1360,1361,2019-04-09,Kishan Gaur,2989363276,CSE-DS,9876545640,868,Conference - Seminar - Workshops - Exhibition ...,Demonstrated dedication and commitment to acad...
615,616,2018-12-19,Kartik Dave,2082639598,CSE-AIML,9876589707,924,External Technical Training Programs,Demonstrated dedication and commitment to acad...
1329,1330,2018-05-14,Vikram Baghel,2968212685,COMPS,9876598801,477,Conference - Seminar - Workshops - Exhibition ...,Received positive feedback from mentors and in...
1896,1897,2019-11-26,Havya Shastri,2659537016,CSE-AIML,9876590624,581,"Membership - IET, IEEE, CSI, ASM",Excellent performance in last semester.


In [64]:
df.pivot_table(index=df['Date'].dt.year, columns='Class', values='Sanctioned Amount', aggfunc='sum').reset_index().fillna(0)


Class,Date,COMPS,CSE-AIML,CSE-DS,EXTC
0,2017,42088,67555,65638,50238
1,2018,70529,66558,81301,72803
2,2019,71686,66141,87847,76490
3,2020,84483,69137,77160,71097
4,2021,68044,71725,66042,74306
5,2022,75168,74492,90252,66592
6,2023,81907,80453,89817,63551
7,2024,13333,26868,7031,36436


In [71]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Student Empowerment Policy', style={'textAlign':'center'}),
    dcc.Dropdown(options=[{'label': i, 'value': i} for i in df.Class.unique()], value=['COMPS'], multi=True, id='dropdown-selection'),

    dcc.Dropdown(
        options=[
            {'label': 'Bar Chart', 'value': 'bar'},
            {'label': 'Pie Chart', 'value': 'pie'},
            {'label': 'Treemap', 'value': 'treemap'}
        ],
        value='bar',  # Set default value
        id='dropdown-graph-type',
        placeholder="Select graph type",
    ),

    dcc.Graph(id='graph-content')
    # 
], style={'width': '50%', 'position': 'relative', 'left': '0'})

@callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value'),
     Input('dropdown-graph-type', 'value')]
)

def update_graph(selected_class, graph_type):
    # Filter DataFrame based on the selected class value
    if(selected_class == []):
        return None

    grouped_class=df.groupby([df.Date.dt.year,'Class'])['Sanctioned Amount'].sum().reset_index()


    # print(df[df['Class'] == selected_class])
    dff = df[df['Class'] == selected_class[0]]

    x_val = dff.Date.dt.year.unique()
    y_val = dff.groupby(dff.Date.dt.year)['Sanctioned Amount'].sum().values



    if graph_type == 'bar':

        data=df[df['Class'].isin(selected_class)].pivot_table(index=df['Date'].dt.year, columns='Class', values='Sanctioned Amount', aggfunc='sum').reset_index().fillna(0)
        
        fig=px.bar(data, x='Date', y=selected_class, title='Sanctioned Amount vs Year', template='plotly_dark',barmode = 'group',)


        return fig

    # elif graph_type == 'line':

    #     fig = px.line(x=x_val, y=y_val, title='Sanctioned Amount vs Year', template='plotly_dark')


    elif graph_type == 'pie':

        specs=[{"type": "pie"} for i in range(len(selected_class))]
        print(specs)
        fig = make_subplots(rows=1, cols=len(selected_class), subplot_titles=selected_class,print_grid=True,specs=[specs])
        
        for i, class_name in enumerate(selected_class, start=1):
            # Filter DataFrame based on the selected class value
            dff = df[df['Class'] == class_name]
            
            # Group by 'Class' and year extracted from 'Date', then sum 'Sanctioned Amount'
            grouped_data = dff.groupby(dff['Date'].dt.year)['Sanctioned Amount'].sum().reset_index()
            labels=grouped_data['Date']
            values=grouped_data['Sanctioned Amount']
            
            # Create a pie chart for the selected class
            trace = go.Pie(labels=labels, values=values, name=class_name)
            fig.add_trace(trace, row=1, col=i)
            
        fig.update_layout(height=400*len(selected_class), title_text="Sanctioned Amount by Year for Selected Classes")
        return fig


    elif graph_type == 'treemap':
        dff['year']=dff['Date'].dt.year
        fig = px.treemap(dff, path=['year'], values='Sanctioned Amount', title='Sanctioned Amount by Scheme', template='ggplot2')


    else:
        fig = None  
    
    return fig

if __name__ == '__main__':
    print("app is running on :" f'http://127.0.0.1:8050/')
    app.run(debug=True)


app is running on :http://127.0.0.1:8050/


C:\Users\Nikhil Prajapati\AppData\Local\Temp\ipykernel_3796\181827067.py:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Nikhil Prajapati\AppData\Local\Temp\ipykernel_3796\181827067.py:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Nikhil Prajapati\AppData\Local\Temp\ipykernel_3796\181827067.py:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h